In [14]:
from bs4 import BeautifulSoup
import json
import os
import sqlite3
import nltk
import pandas as pd
import numpy as np
import re
# from gensim.corpora.dictionary import Dictionary
# from gensim.models.phrases import Phrases
# from gensim.models import Word2Vec

In [16]:
conn = sqlite3.connect("../../diplom.db")
cur = conn.cursor()

In [18]:
res = cur.execute("""SELECT * FROM IZVESTIA_CLEANTEXTS0021 ORDER BY DEMOCRACY_COUNT DESC LIMIT 100;""")

In [19]:
for row in res:
    print(row[2], row[6])

2021-03-31 12
2021-07-19 11
2020-10-04 10
2005-02-24 8
2020-07-07 8
2019-01-26 7
2017-10-05 7
2021-04-12 6
2021-01-20 6
2018-04-18 5
2019-01-30 5
2017-06-08 5
2021-03-15 5
2019-01-31 5
2020-06-06 5
2018-07-17 4
2021-03-25 4
2021-08-24 4
2021-05-25 4
2020-09-03 4
2021-01-11 4
2021-02-01 4
2018-02-20 4
2020-06-16 4
2019-08-14 4
2021-01-08 4
2021-02-01 4
2018-08-13 4
2019-12-02 4
2019-03-13 4
2021-01-06 4
2018-04-14 4
2017-09-25 4
2021-07-22 3
2020-02-25 3
2019-06-03 3
2018-12-10 3
2020-05-03 3
2020-09-16 3
2020-04-05 3
2021-03-24 3
2020-03-22 3
2018-08-20 3
2019-03-23 3
2019-03-07 3
2020-01-23 3
2020-11-10 3
2017-09-03 3
2019-01-27 3
2017-09-29 3
2020-10-18 3
2020-03-28 3
2018-06-28 3
2020-08-19 3
2021-02-19 3
2017-12-12 3
2020-02-13 3
2018-07-14 3
2017-09-13 3
2018-09-08 3
2018-04-01 3
2019-06-21 3
2021-08-17 3
2018-05-12 3
2021-01-07 3
2017-09-26 3
2019-07-27 3
2020-09-30 3
2021-02-10 3
2020-12-01 3
2021-02-02 3
2019-01-24 2
2020-09-10 2
2021-01-19 2
2021-07-15 2
2017-11-16 2
2021-09-0

In [13]:
!pip uninstall gensim

^C


In [ ]:
!pip install gensim

In [ ]:
phrases = Phrases(corpus_izvestia, min_count=1, threshold=1)

In [8]:
model.get_vector('демократия')

array([-3.5766926 , -3.0316958 ,  0.9686452 , -1.3188351 ,  2.546759  ,
       -0.6474348 ,  0.09102358, -0.45753068, -0.70596963,  1.4003515 ,
        0.46409538,  2.400015  , -0.3453278 , -1.0255783 , -1.0174031 ,
       -0.76842135,  1.3332396 ,  0.08861988,  0.92493194, -2.474383  ,
       -1.0612268 ,  2.1174538 ,  1.8337611 , -0.6436659 , -0.47902954,
        0.71494097,  0.46093917,  1.6548154 ,  1.2463144 , -0.66734   ,
       -2.7372336 ,  2.6495342 ], dtype=float32)

In [9]:
model.most_similar('демократия')

[('ценность', 0.8455156087875366),
 ('справедливость', 0.8448948860168457),
 ('народ', 0.794706404209137),
 ('принцип', 0.783704400062561),
 ('общество', 0.7612237930297852),
 ('либеральный', 0.7468475699424744),
 ('свобода', 0.7453266978263855),
 ('меньшинство', 0.7419408559799194),
 ('движение', 0.7358075380325317),
 ('нация', 0.7312542796134949)]

In [26]:
model = Word2Vec(corpus_izvestia, 
                 vector_size=64,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=2).wv

In [27]:
model.most_similar('демократия')

[('ценность', 0.8228520154953003),
 ('принцип', 0.7854652404785156),
 ('справедливость', 0.7762071490287781),
 ('либеральный', 0.7531120181083679),
 ('меньшинство', 0.7300997972488403),
 ('общество', 0.7276099324226379),
 ('единство', 0.7206049561500549),
 ('цивилизация', 0.7091200351715088),
 ('свобода', 0.7019317150115967),
 ('движение', 0.6981418132781982)]

In [42]:
model = Word2Vec(corpus_izvestia, 
                 vector_size=1024,      # embedding vector size
                 min_count=1,  # consider words that occured at least 5 times
                 window=3).wv

In [43]:
model.most_similar('демократия')

[('ценность', 0.8878699541091919),
 ('общество', 0.8470258116722107),
 ('принцип', 0.8460907340049744),
 ('справедливость', 0.845017671585083),
 ('народ', 0.823380708694458),
 ('стандарт', 0.8123600482940674),
 ('движение', 0.8115740418434143),
 ('либеральный', 0.7893827557563782),
 ('меньшинство', 0.783093273639679),
 ('свобода', 0.7830301523208618)]